In [ ]:
from stmpy import Driver, Machine

import ipywidgets as widgets
from IPython.display import display

tasks = [1, 2, 3, 4, 5]

class Student:
    def __init__(self):
        print("Init - Student")
                
        self.button_next_task = widgets.Button(description="Next task")
        self.button_next_task.on_click(self.next_task_pressed)

        self.button_help = widgets.Button(description="Help")
        self.button_help.on_click(self.help_pressed)

        self.counter = 0
        display(self.button_next_task)
        display(self.button_help)

    def show_tasks(self):
        self.stm.send('start_task')

    def task_func(self):
        if self.button_next_task.description == "Next task":
            if self.counter == len(tasks) - 1:
                self.button_next_task.description = "Submit tasks"
            else:
                self.button_next_task.description = "Next task"
            print("Show tasks nr: ", tasks[self.counter])
        else:
            print("Delivered")
            self.stm.send('deliver')

    def next_task_pressed(self, b):
        self.counter += 1
        self.stm.send('next_task')

    def help_pressed(self, b):
        if self.button_help.description == "Help":
            self.button_help.description = "Help finished"
            self.stm.send('help')
        else:
            self.button_help.description = "Help"
            self.stm.send('help_finished')
    
       
    def add_group_to_queue(self):
        print("Lagt til i kø")
        self.button_help.description = "Help finished"

    def finished(self):
        self.stm.driver.stop()


    #def ask_question(self):
        #print("What is 1+1?")
        #self.mqtt_client.publish("question", "What is 1+1?")

    #def send_mqtt_tock(self):
        #print("Tock! {}".format(self.tocks)) 
        #self.tocks = self.tocks + 1
        #self.mqtt_client.publish("group16 tock", self.ticks)


#for student
s0 = {"source": "initial", "target": "display_tasks"}

s1 = {
    "trigger": "start_task",
    "source": "display_tasks",
    "target": "task",
}
s2 = {
    "trigger": "next_task",
    "source": "task",
    "target": "task",
}
s3 = {
    "trigger": "help",
    "source": "task",
    "target": "TA_assistance",
}
s4 = {
    "trigger": "t",
    "source": "task",
    "target": "TA_assistance",
}
s5 = {
    "trigger": "deliver",
    "source": "task",
    "target": "end", #what is end?
}
s6 = {
    "trigger": "back_to_waiting",
    "source": "task",
    "target": "display_tasks",
}
s7 = {
    "trigger": "help_finished",
    "source": "TA_assistance",
    "target": "task",
}


#student states
display_tasks = {'name': 'display_tasks',
      'entry': 'show_tasks'
      }
task = {'name': 'task',
      'entry': 'start_timer("t", 10000); task_func'
      }
TA_assistance = {'name': 'TA_assistance',
      'entry': 'stop_timer("t"); add_group_to_queue'
      }
end = {'name': 'end',
      'entry': 'finished'
      }


student = Student()
student_machine = Machine(name='stm_student', transitions=[s0, s1, s2, s3, s4, s5, s6, s7], obj=student, states=[display_tasks, task, TA_assistance, end])
student.stm = student_machine

driver = Driver()
driver.add_machine(student_machine)
driver.start()

#driver.stop()